In [64]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("hourlySummaries.csv")
data = data.set_index("hour")
data = data[data.columns[1:]]
# data = data.iloc[:1000,]

train_weight = 0.80
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [51]:
X_train = df_train[["numBorrows","amountBorrowedUSD"]]
X_test = df_test[["numBorrows","amountBorrowedUSD"]]
y_train = df_train["aaveHourlyChange"]
y_test = df_test["aaveHourlyChange"]

In [52]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [53]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [54]:
keras.backend.clear_session()

In [55]:
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(2, 1)))
model.add(keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(x_train_reg, y_train_reg, epochs=10, validation_split=0.2, verbose=1)

y_pred = model.predict(x_test_reg)

Train on 6944 samples, validate on 1736 samples
Epoch 1/10
6944/6944 [==============================] - 4s 592us/step - loss: 1495825056.8489 - val_loss: 1157900685.6774
Epoch 2/10
6944/6944 [==============================] - 3s 378us/step - loss: 81260060.6684 - val_loss: 31953.2548
Epoch 3/10
6944/6944 [==============================] - 3s 373us/step - loss: 71405.7871 - val_loss: 615261.6202
Epoch 4/10
6944/6944 [==============================] - 3s 411us/step - loss: 24653026.4202 - val_loss: 28991.8086
Epoch 5/10
6944/6944 [==============================] - 3s 375us/step - loss: 440001477.1605 - val_loss: 29071323645.6406
Epoch 6/10
6944/6944 [==============================] - 3s 403us/step - loss: 40781618005.8365 - val_loss: 74005804.2350
Epoch 7/10
6944/6944 [==============================] - 3s 368us/step - loss: 4975921.4390 - val_loss: 10.7498
Epoch 8/10
6944/6944 [==============================] - 3s 476us/step - loss: 31.0231 - val_loss: 10.7284
Epoch 9/10
6944/6944 [=====

# Alpha RNN

In [56]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [57]:
max_epoches=2000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

# params = {'alpharnn': {'model':'', 'function':AlphaRNN_,'l1_reg':0.0, 'H':10,'color': 'green', 'label': '$\\alpha$-RNN' }}

In [58]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=10, batch_size=batch_size,callbacks=[es],shuffle=False)

# for key in params.keys():
#     tf.random.set_seed(0)
#     # model=params[key]['function'](params[key]['H'],params[key]['l1_reg'])
#     model = AlphaRNN_(10,0.0)
#     model.fit(x_train_reg,y_train_reg,epochs=10, batch_size=batch_size,callbacks=[es],shuffle=False)
#     params[key]['model']=model

Epoch 1/10
8680/8680 [==============================] - 1s 100us/step - loss: 27.1148
Epoch 2/10
8680/8680 [==============================] - 0s 7us/step - loss: 27.0243
Epoch 3/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.9457
Epoch 4/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.8796
Epoch 5/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.8254
Epoch 6/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.7823
Epoch 7/10
8680/8680 [==============================] - 0s 7us/step - loss: 26.7488
Epoch 8/10
8680/8680 [==============================] - 0s 7us/step - loss: 26.7235
Epoch 9/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.7048
Epoch 10/10
8680/8680 [==============================] - 0s 6us/step - loss: 26.6913


In [63]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_1 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [66]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)


2171/2171 [==============================] - 0s 21us/step
